In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
!pip install -q streamlit

     |████████████████████████████████| 7.0MB 5.5MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
     |████████████████████████████████| 112kB 50.5MB/s 
     |████████████████████████████████| 4.6MB 46.3MB/s 
     |████████████████████████████████| 163kB 55.9MB/s 
     |████████████████████████████████| 122kB 55.0MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.3 which is incompatible.


In [4]:
import requests
import pickle
from datetime import datetime
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import streamlit as st
#use the below library while displaying the images in jupyter notebook
from IPython.display import display, Imag

ContextualVersionConflict: ignored

In [ ]:
import numpy as np                # mathematical calculations
import pandas as pd               # manipulation of raw data 
import matplotlib.pyplot as plt   # plotting graphs
%matplotlib inline 

In [ ]:
path = '/gdrive/My Drive/Aegis_Capstone_Project/'

In [ ]:
final_jobs = pd.read_csv(path+'Combined_Jobs_Final.csv')
final_jobs.head()

In [ ]:
final_jobs.shape

In [ ]:
# Listing out all the columns that are present in the data set.

list(final_jobs) 

In [ ]:
print(final_jobs.shape)
final_jobs.isnull().sum()

 From the above list we see that there are lot of NaN values, perform data cleansing for each and every column

## Concatenating the columns ( job corups)


In [ ]:
#subsetting only needed columns and not considering the columns that are not necessary
cols = list(['Job.ID']+['Slug']+['Title']+['Position']+ ['Company']+['City']+['Employment.Type']+['Education.Required']+['Job.Description'])
final_jobs =final_jobs[cols]
final_jobs.columns = ['Job.ID','Slug', 'Title', 'Position', 'Company','City', 'Empl_type','Edu_req','Job_Description']
final_jobs.head() 

In [ ]:
# checking for the null values again.
final_jobs.isnull().sum()

In [ ]:
final_jobs.shape

In [ ]:
#selecting NaN rows of city
nan_city = final_jobs[pd.isnull(final_jobs['City'])]
print(nan_city.shape)
nan_city.head()

In [ ]:
nan_city.groupby(['Company'])['City'].count() 

We see that there are only 9 companies cities that are having NaN values so I manually adding their head quarters with the help of google search


In [ ]:
#replacing nan with thier headquarters location
final_jobs['Company'] = final_jobs['Company'].replace(['Genesis Health Systems'], 'Genesis Health System')

final_jobs.loc[final_jobs.Company == 'CHI Payment Systems', 'City'] = 'Illinois'
final_jobs.loc[final_jobs.Company == 'Academic Year In America', 'City'] = 'Stamford'
final_jobs.loc[final_jobs.Company == 'CBS Healthcare Services and Staffing ', 'City'] = 'Urbandale'
final_jobs.loc[final_jobs.Company == 'Driveline Retail', 'City'] = 'Coppell'
final_jobs.loc[final_jobs.Company == 'Educational Testing Services', 'City'] = 'New Jersey'
final_jobs.loc[final_jobs.Company == 'Genesis Health System', 'City'] = 'Davennport'
final_jobs.loc[final_jobs.Company == 'Home Instead Senior Care', 'City'] = 'Nebraska'
final_jobs.loc[final_jobs.Company == 'St. Francis Hospital', 'City'] = 'New York'
final_jobs.loc[final_jobs.Company == 'Volvo Group', 'City'] = 'Washington'
final_jobs.loc[final_jobs.Company == 'CBS Healthcare Services and Staffing', 'City'] = 'Urbandale'

In [ ]:
final_jobs.isnull().sum()

In [ ]:
#The employement type NA are from Uber so I assume as part-time and full time
nan_emp_type = final_jobs[pd.isnull(final_jobs['Empl_type'])]
print(nan_emp_type)



In [ ]:
#replacing na values with part time/full time
final_jobs['Empl_type']=final_jobs['Empl_type'].fillna('Full-Time/Part-Time')
final_jobs.groupby(['Empl_type'])['Company'].count()
list(final_jobs)

#   corpus 

#### combining the columns of position,company,city,emp_type and position

In [ ]:
final_jobs["pos_com_city_empType_jobDesc"] = final_jobs["Position"].map(str) + " " + final_jobs["Company"] +" "+ final_jobs["City"]+ " "+final_jobs['Empl_type']+" "+final_jobs['Job_Description']
final_jobs.pos_com_city_empType_jobDesc.head()

In [ ]:
#removing unnecessary characters between words separated by space between each word of all columns to make the data efficient
final_jobs['pos_com_city_empType_jobDesc'] = final_jobs['pos_com_city_empType_jobDesc'].str.replace('[^a-zA-Z \n\.]'," ") #removing unnecessary characters
final_jobs.pos_com_city_empType_jobDesc.head()

In [ ]:
#converting all the characeters to lower case
final_jobs['pos_com_city_empType_jobDesc'] = final_jobs['pos_com_city_empType_jobDesc'].str.lower() 
final_jobs.pos_com_city_empType_jobDesc.head()

In [ ]:
final_all = final_jobs[['Job.ID', 'pos_com_city_empType_jobDesc']]
# renaming the column name as it seemed a bit complicated
final_all = final_jobs[['Job.ID', 'pos_com_city_empType_jobDesc']]
final_all = final_all.fillna(" ")

final_all.head()

The next package used in here is stemming, The idea of stemming is a sort of normalizing method. Many variations of words carry the same meaning, other than when tense is involved. So in order to clean up the space, we use stemming method and the one of the packages used here is PorterStemmer

In [ ]:
print(final_all.head(1))

  ##  NLTK Process
It removes stop words such as the, is, and etc..

In [ ]:
# Setup
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

import pandas as pd
import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string

In [ ]:
pos_com_city_empType_jobDesc = final_all['pos_com_city_empType_jobDesc']
#removing stopwords and applying potter stemming
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer =  PorterStemmer()
stop = stopwords.words('english')
only_text = pos_com_city_empType_jobDesc.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
only_text.head()


Splitting each word in a row separated by space.

In [ ]:
only_text = only_text.apply(lambda x : filter(None,x.split(" ")))
print(only_text.head())

Here **stemming** is basically used to remove the suffixes and common words that repeat and separated by commas. for y in x means, for each word(y) in the total list(x)

In [ ]:
only_text = only_text.apply(lambda x : [stemmer.stem(y) for y in x])
print(only_text.head())

In the above code, we separated each letter in a word separated by comma, now, in this step, we join the words(x)


In [ ]:
only_text = only_text.apply(lambda x : " ".join(x))
print(only_text.head())

In [ ]:
#adding the featured column back to pandas
final_all['text']= only_text
# As we have added a new column by performing all the operations using lambda function, we are removing the unnecessary column
#final_all = final_all.drop("pos_com_city_empType_jobDesc", 1)

list(final_all)
final_all.head()

In [ ]:
# in order to save this file for a backup
#final_all.to_csv("job_data.csv", index=True)

# TF-IDF ( Term Frequency - Inverse Document Frequency ) 
This method is also called as Normalization.
TF - How many times a particular word appears in a single doc. 
IDF - This downscales words that appear a lot across documents.

There is a difference between fit (unique words are created / vectorization) and fit_transform.
fit means fit transform and transform( adding the row internally adding 1 in the users input query)
ex : innovate is not present then then it will not create ( loosing the efficiency)

In [ ]:

#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer

tfidf_vectorizer = TfidfVectorizer()

tfidf_jobid = tfidf_vectorizer.fit_transform((final_all['text'])) #fitting and transforming the vector
tfidf_jobid

# User query Corpus
Take another dataset called job views.

In [ ]:
#Consider a  new data set and  taking the datasets job view, position of interest, experience of the applicant into consideration for creating a query who applied for job
job_view = pd.read_csv(path+'Job_Views.csv')
job_view.head()


In [ ]:
#subsetting only needed columns and not considering the columns that are not necessary as we did that earlier.
job_view = job_view[['Applicant.ID', 'Job.ID', 'Position', 'Company','City']]

job_view["pos_com_city"] = job_view["Position"].map(str) + "  " + job_view["Company"] +"  "+ job_view["City"]

job_view['pos_com_city'] = job_view['pos_com_city'].str.replace('[^a-zA-Z \n\.]',"")

job_view['pos_com_city'] = job_view['pos_com_city'].str.lower()

job_view = job_view[['Applicant.ID','pos_com_city']]

job_view.head()


### Experience
We take experience of all the applicants who applied for the job and we are comaring the point of interest with the jobs that are present in our previous data.

In [ ]:
#Experience
exper_applicant = pd.read_csv(path+'Experience.csv')
exper_applicant.head()

In [ ]:
#taking only Position
exper_applicant = exper_applicant[['Applicant.ID','Position.Name']]

#cleaning the text
exper_applicant['Position.Name'] = exper_applicant['Position.Name'].str.replace('[^a-zA-Z \n\.]',"")

exper_applicant.head()
#list(exper_applicant)

In [ ]:
exper_applicant['Position.Name'] = exper_applicant['Position.Name'].str.lower()
exper_applicant.head(10)

In [ ]:
exper_applicant =  exper_applicant.sort_values(by='Applicant.ID')
exper_applicant = exper_applicant.fillna(" ")
exper_applicant.head(20)


same applicant has 3 applications 100001 in sigle line

In [ ]:
#adding same rows to a single row
exper_applicant = exper_applicant.groupby('Applicant.ID', sort=False)['Position.Name'].apply(' '.join).reset_index()
exper_applicant.head(20)

### Position of Interest

In [ ]:
#Position of interest
poi =  pd.read_csv(path+'Positions_Of_Interest.csv')
poi = poi.sort_values(by='Applicant.ID')
poi.head()

In [ ]:
# There is no need of application and updation becuase there is no deadline mentioned in the website ( assumption) hence we are droping unimportant attributes
poi = poi.drop('Updated.At', 1)
poi = poi.drop('Created.At', 1)

#cleaning the text
poi['Position.Of.Interest']=poi['Position.Of.Interest'].str.replace('[^a-zA-z \n\.]',"")
poi['Position.Of.Interest']=poi['Position.Of.Interest'].str.lower()
poi = poi.fillna(" ")
poi.head(20)

In [ ]:
poi = poi.groupby('Applicant.ID', sort=True)['Position.Of.Interest'].apply(' '.join).reset_index()
poi.head()

### Merging

pos_com city, position_name

In [ ]:
#merging jobs and experience dataframes
out_joint_jobs = job_view.merge(exper_applicant, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
print(out_joint_jobs.shape)
out_joint_jobs = out_joint_jobs.fillna(' ')
out_joint_jobs = out_joint_jobs.sort_values(by='Applicant.ID')
out_joint_jobs.head()

In [ ]:
#merging position of interest with existing dataframe
joint_poi_exper_view = out_joint_jobs.merge(poi, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
joint_poi_exper_view = joint_poi_exper_view.fillna(' ')
joint_poi_exper_view = joint_poi_exper_view.sort_values(by='Applicant.ID')
joint_poi_exper_view.head()

In [ ]:
#combining all the columns

joint_poi_exper_view["pos_com_city1"] = joint_poi_exper_view["pos_com_city"].map(str) + joint_poi_exper_view["Position.Name"] +" "+ joint_poi_exper_view["Position.Of.Interest"]

joint_poi_exper_view.head()

In [ ]:
final_poi_exper_view = joint_poi_exper_view[['Applicant.ID','pos_com_city1']]
final_poi_exper_view.head()

In [ ]:
final_poi_exper_view.columns = ['Applicant_id','pos_com_city1']
final_poi_exper_view.head()

In [ ]:
final_poi_exper_view = final_poi_exper_view.sort_values(by='Applicant_id')
final_poi_exper_view.head()

In [ ]:
final_poi_exper_view['pos_com_city1'] = final_poi_exper_view['pos_com_city1'].str.replace('[^a-zA-Z \n\.]',"")
final_poi_exper_view.head()


In [ ]:
final_poi_exper_view['pos_com_city1'] = final_poi_exper_view['pos_com_city1'].str.lower()
final_poi_exper_view.head()

In [ ]:

final_poi_exper_view = final_poi_exper_view.reset_index(drop=True)
final_poi_exper_view.head()

select random row of 6999

In [ ]:
#taking a user
u = 6999
index = np.where(final_poi_exper_view['Applicant_id'] == u)[0][0]
user_q = final_poi_exper_view.iloc[[index]]
user_q

# Using vector space model ( Cosine similarity )
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [ ]:
#creating tf-idf of user query and computing cosine similarity of user with job corpus
from sklearn.metrics.pairwise import cosine_similarity
user_tfidf = tfidf_vectorizer.transform(user_q['pos_com_city1'])
output = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_jobid)


In [ ]:
output2 = list(output)

In [ ]:
import sys

In [ ]:
#getting the job id's of the recommendations
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:50]
recommendation = pd.DataFrame(columns = ['ApplicantID', 'JobID'])
count = 0
for i in top:
  recommendation.at[count, 'ApplicantID']=u
  recommendation.at[count,'JobID'] = final_all['Job.ID'][i]
  count += 1

cosine similarity score

In [ ]:
recommendation

In [ ]:
#getting the job ids and their data
nearestjobs = recommendation['JobID']
job_description = pd.DataFrame(columns = ['JobID','text'])
for i in nearestjobs:
    index = np.where(final_all['Job.ID'] == i)[0][0]    
    job_description.at[count, 'JobID']=i
    job_description.at[count, 'text']= final_all['text'][index]
    count += 1
    

In [ ]:
#printing the jobs that matched the query
job_description

In [ ]:
job_description.to_csv("recommended_content.csv")

In [ ]:
final_all.to_csv("job_data.csv", index=False)